In [1]:
from datetime import datetime
from os import scandir
#import triangles as tr
import csvfile as csv
import profits as pr
import csvfile
import time
from binance.client import Client

In [2]:
api_key = "ucqMzmbqswWPqo4bfQp4DoPyUTFIV2wO749sXmAPyIqwkA4vKQKSO6CZKAPi3DEA"
api_secret = "KRQYhVnepPwutIprQmxyDbugCGCNMkr7WL3Cm7zqQoK9GIuGdLJQpoJqIooTqWnY"

In [3]:
try:
    client = Client(api_key, api_secret)
except BinanceAPIException as e:
    print(e.status_code)
    print(e.message)
    print(e.code)

In [4]:
# This master list collects dictionaries - pair symbols with base and quote assets
my_pairs = []

In [5]:
def load_pairs():

    ### STEP 1: Load all Binance pairs

    all_tickers = client.get_all_tickers()

    for info in all_tickers:
        my_pairs.append(dict(symbol=info['symbol']))
        
    print(len(my_pairs), 'Binance pairs found and loaded.')

In [6]:
def update_pairs():

    ### STEP 2: for each pair call Binance again for quote+asset base and status, and add to my_pairs
    ##Risk here - breaching 1200 api calls per minute.....
    ##Risk here - some pairs return None type - ask Binance why :)

    lag = 0.1 # lag for not hitting Binance API 1200/min threshold
    
    print('Loading base and quote currencies, and statuses...')
    for i in range(len(my_pairs)):
        time.sleep(lag)
        details = client.get_symbol_info(my_pairs[i]['symbol'])
        if details is not None:
            my_pairs[i]['baseAsset'] = details['baseAsset']
            my_pairs[i]['quoteAsset'] = details['quoteAsset']
            my_pairs[i]['status'] = details['status']
            if i % 500 == 0: print(time.ctime(),'|| Loaded:',i)
    print('Finished loading base and quote assets, and status.')
    csvfile.lod_to_csv(my_pairs,'my_pairs.csv')

In [7]:
def get_file_info(name):
    dir_entries = scandir('./')
    for entry in dir_entries:
        if entry.is_file() and entry.name==name:
            info = entry.stat()
    return info.st_mtime

In [8]:
def convert_date(timestamp):
    d = datetime.utcfromtimestamp(timestamp)
    formated_date = d.strftime('%d %b %Y')
    return formated_date

In [9]:
# Check last update time for 'my_pairs.csv', 'my_triangles.csv' and 'my_profits.csv'

print('**************************************')
print(f'my_pairs.csv\t\t was last modified: {convert_date(get_file_info("my_pairs.csv"))}')
print(f'my_triangles.csv\t was last modified: {convert_date(get_file_info("my_triangles.csv"))}')
print(f'my_profits.csv\t\t was last modified: {convert_date(get_file_info("my_profits.csv"))}')

**************************************
my_pairs.csv		 was last modified: 06 Jan 2023
my_triangles.csv	 was last modified: 06 Jan 2023
my_profits.csv		 was last modified: 09 Jan 2023


In [10]:
# Interface - are updates needed?

#choice = input("\nDo you want to update my_pairs (~21 mins!) y/n ?")
#if choice=="y": pa.load_pairs(); pa.update_pairs()

#choice = input("\nDo you want to update my_triangles (~4 mins!) y/n ?")
#if choice=="y": tr.find_triangles()

In [11]:
print("\nBest potential trades")
print("**********************************")


Best potential trades
**********************************


In [12]:
all_profits = pr.find_profits()

Mon Jan  9 16:29:01 2023 || Starting search...
Mon Jan  9 16:29:02 2023 || Finished search. Found 2430 profits.


In [13]:
buy_buy_sell = pr.sort_profits(all_profits, 6)
buy_sell_sell = pr.sort_profits(all_profits, 11)

print("\nTop 5 for Buy-Buy-Sell")
for i in range(5):
    print(buy_buy_sell[i])

print("\nTop 5 for Buy-Sell-Sell")
for i in range(5):
    print(buy_sell_sell[i])


Top 5 for Buy-Buy-Sell
['BNB', 'XRP', 'RUB', 19.79067, 19.75784967, -0.03282033, -0.0017, 'RUB', 21292.92, 21182.49290334, -110.42709666, -0.0052, 'RUB']
['BNB', 'SOL', 'RUB', 19.79067, 19.75482332, -0.03584668, -0.0018, 'RUB', 60494.25, 60157.86063442, -336.38936558, -0.0056, 'RUB']
['BTC', 'ZEN', 'USDT', 8133.375789, 8117.84250872, -15.53328028, -0.0019, 'USDT', 1054.0724, 1048.08311993, -5.98928007, -0.0057, 'USDT']
['BTC', 'ONT', 'BUSD', 34.48444, 34.41491246, -0.06952754, -0.002, 'BUSD', 387.1835, 385.59571606, -1.58778394, -0.0041, 'BUSD']
['BTC', 'MOB', 'USDT', 8133.375789, 8117.46464202, -15.91114698, -0.002, 'USDT', 458.04196, 455.91601158, -2.12594842, -0.0046, 'USDT']

Top 5 for Buy-Sell-Sell
['ETH', 'AXS', 'BIDR', 2361401.928, 2293149.74667475, -68252.18132525, -0.0289, 'BIDR', 115.555, 116.02664705, 0.47164705, 0.0041, 'BIDR']
['BUSD', 'AXS', 'BIDR', 4580077.2, 4449093.91377481, -130983.28622519, -0.0286, 'BIDR', 115.555, 115.95145194, 0.39645194, 0.0034, 'BIDR']
['USDT',